In [6]:
###### First attempt to conduct machine learning on text data within the student evals 
## install pkgs 
import sys
import os
# !{sys.executable} -m pip install xgboost==1.7.5 # note: needed since it looks like anaconda installs an earlier version 
# of the package, which is not helpful. 1.7.5 allows for the categorical data of interest to be used. 

# !{sys.executable} -m pip install requests #; this code here can be used to install packages on anaconda/jupyter notebook 
### I believe the below should be installed by default 
import requests # web scraping 
from bs4 import BeautifulSoup # for web scraping 
import itertools # for efficient operation of loops 
import pandas as pd # necessary for reading in, creating, and manipulating data frames 
import csv ## for importing/exporting csvs 
import glob ## for finding files in path
import re
import numpy as np
### THe ML packages 
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# sklearn packages
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [54]:
#### read in the data 
rmp_df = pd.read_csv("text_cleaning_data/scored_rmp_data06212023.csv")
rmp_df

,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,out_emo_lang,...,pb_origin,pb_nuero_div,pb_phys_able,pb_pol_affil,complex,constructive,reflective,outrage_agg,personal_attack_agg,prejudice_agg
0,1,2.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,"Only graded on 4 assignments (30% Midterm, 30%...",0,0.0,...,0.0,0,0,0.0,0,0,0,0.0,0,0.0
1,2,3.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,"Final grade is only based on two exams, readin...",0,0.0,...,0.0,0,0,0.0,1,0,0,0.0,0,0.0
2,3,4.0,1,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy. One reading quiz a week ...,0,0.0,...,0.0,0,0,0.0,0,0,0,0.0,0,0.0
3,4,4.0,2,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,"Lecture could be dry at times, but I still lik...",0,0.0,...,0.0,0,0,0.0,0,0,1,0.0,0,0.0
4,5,5.0,3,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,"Alex was an excellent lecturer. Insightful, ev...",0,0.0,...,0.0,0,0,0.0,0,0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,4,4.0,4,SOC410,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,She's very fair. I got away with calling Karl ...,0,0.0,...,0.0,0,0,0.0,0,0,0,0.0,0,0.0
1096,5,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Professor klochko is cool and all but her acce...,0,0.0,...,0.0,0,0,0.0,0,0,0,0.0,0,0.0
1097,6,1.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,"Definitely not ""the greatest thing since slice...",0,0.0,...,0.0,0,0,0.0,0,0,0,1.0,0,0.0
1098,7,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,"She's not personal, almost to a harsh point. T...",0,0.0,...,0.0,0,0,0.0,0,0,0,1.0,1,0.0


In [55]:
### now clean the text 
regex = r"[\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]" # I removed ! from the beginning 

rmp_df['comment'] = rmp_df['comment'].replace(to_replace =regex, value = '', regex = True)
rmp_df['comment'] = rmp_df['comment'].replace(to_replace ="\d+", value = '', regex = True)

#rmp_df['comment'] = re.sub(regex, '', str(rmp_df['comment']).strip() )
#rmp_df['comment'] =  re.sub("\d+", " ", str(rmp_df['comment'] ))

## reading in the stopwords 
stop_words = stopwords.words('english') # creates a list of English stop words
# wnl = WordNetLemmatizer() # I used lemmatizing instead of stemming
 ## looks like it worked now 

pat = '|'.join([r'\b{}\b'.format(w) for w in stop_words])

rmp_df['comment_edit'] = rmp_df['comment'].str.replace(pat, '') ## good, stop words removed 
rmp_df['comment_edit'] = rmp_df['comment_edit'].str.strip()
### now apply stemming

rmp_df

C:\Users\j-curiel\AppData\Local\Temp\ipykernel_11280\3636061428.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  rmp_df['comment_edit'] = rmp_df['comment'].str.replace(pat, '') ## good, stop words removed


,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,out_emo_lang,...,pb_nuero_div,pb_phys_able,pb_pol_affil,complex,constructive,reflective,outrage_agg,personal_attack_agg,prejudice_agg,comment_edit
0,1,2.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Only graded on assignments Midterm Final R...,0,0.0,...,0,0,0.0,0,0,0,0.0,0,0.0,Only graded assignments Midterm Final Rea...
1,2,3.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Final grade is only based on two exams reading...,0,0.0,...,0,0,0.0,1,0,0,0.0,0,0.0,Final grade based two exams reading quizzes...
2,3,4.0,1,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy One reading quiz a week w...,0,0.0,...,0,0,0.0,0,0,0,0.0,0,0.0,Class super easy One reading quiz week com...
3,4,4.0,2,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Lecture could be dry at times but I still like...,0,0.0,...,0,0,0.0,0,0,1,0.0,0,0.0,Lecture could dry times I still liked clas...
4,5,5.0,3,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Alex was an excellent lecturer Insightful even...,0,0.0,...,0,0,0.0,0,0,0,0.0,0,0.0,Alex excellent lecturer Insightful even hand...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,4,4.0,4,SOC410,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes very fair I got away with calling Karl Ma...,0,0.0,...,0,0,0.0,0,0,0,0.0,0,0.0,Shes fair I got away calling Karl Marx idio...
1096,5,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Professor klochko is cool and all but her acce...,0,0.0,...,0,0,0.0,0,0,0,0.0,0,0.0,Professor klochko cool accent makes hard...
1097,6,1.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Definitely not the greatest thing since slice ...,0,0.0,...,0,0,0.0,0,0,0,1.0,0,0.0,Definitely greatest thing since slice bread ...
1098,7,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes not personal almost to a harsh point The ...,0,0.0,...,0,0,0.0,0,0,0,1.0,1,0.0,Shes personal almost harsh point The class ...


In [56]:
### now apply stemming to the comments 
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 
rmp_df_sub = rmp_df.iloc[0:2,:]
#for w in rmp_df_sub.comment_edit: 
#    print(w, " : ", ps.stem(w)) 
rmp_df['stem'] = rmp_df.comment_edit.apply(ps.stem)
    
rmp_df    

,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,out_emo_lang,...,pb_phys_able,pb_pol_affil,complex,constructive,reflective,outrage_agg,personal_attack_agg,prejudice_agg,comment_edit,stem
0,1,2.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Only graded on assignments Midterm Final R...,0,0.0,...,0,0.0,0,0,0,0.0,0,0.0,Only graded assignments Midterm Final Rea...,only graded assignments midterm final rea...
1,2,3.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Final grade is only based on two exams reading...,0,0.0,...,0,0.0,1,0,0,0.0,0,0.0,Final grade based two exams reading quizzes...,final grade based two exams reading quizzes...
2,3,4.0,1,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy One reading quiz a week w...,0,0.0,...,0,0.0,0,0,0,0.0,0,0.0,Class super easy One reading quiz week com...,class super easy one reading quiz week com...
3,4,4.0,2,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Lecture could be dry at times but I still like...,0,0.0,...,0,0.0,0,0,1,0.0,0,0.0,Lecture could dry times I still liked clas...,lecture could dry times i still liked clas...
4,5,5.0,3,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Alex was an excellent lecturer Insightful even...,0,0.0,...,0,0.0,0,0,0,0.0,0,0.0,Alex excellent lecturer Insightful even hand...,alex excellent lecturer insightful even hand...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,4,4.0,4,SOC410,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes very fair I got away with calling Karl Ma...,0,0.0,...,0,0.0,0,0,0,0.0,0,0.0,Shes fair I got away calling Karl Marx idio...,shes fair i got away calling karl marx idio...
1096,5,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Professor klochko is cool and all but her acce...,0,0.0,...,0,0.0,0,0,0,0.0,0,0.0,Professor klochko cool accent makes hard...,professor klochko cool accent makes hard...
1097,6,1.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Definitely not the greatest thing since slice ...,0,0.0,...,0,0.0,0,0,0,1.0,0,0.0,Definitely greatest thing since slice bread ...,definitely greatest thing since slice bread ...
1098,7,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes not personal almost to a harsh point The ...,0,0.0,...,0,0.0,0,0,0,1.0,1,0.0,Shes personal almost harsh point The class ...,shes personal almost harsh point the class ...


In [78]:
### attempt to lemmatize, thereby shortening words 
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

## lematize here 
text_lemmatized = rmp_df.comment_edit.apply(lemmatize_text)

In [79]:
#rmp_df.assign(text_lemmatized=text_lemmatized)
rmp_df['text_lemmatized'] = pd.Series(text_lemmatized)

In [80]:
print(rmp_df['text_lemmatized'])

0       [Only, graded, assignment, Midterm, Final, Rea...
1       [Final, grade, based, two, exam, reading, quiz...
2       [Class, super, easy, One, reading, quiz, week,...
3       [Lecture, could, dry, time, I, still, liked, c...
4       [Alex, excellent, lecturer, Insightful, even, ...
                              ...                        
1095    [Shes, fair, I, got, away, calling, Karl, Marx...
1096    [Professor, klochko, cool, accent, make, hard,...
1097    [Definitely, greatest, thing, since, slice, br...
1098    [Shes, personal, almost, harsh, point, The, cl...
1099    [Klochko, unpopular, Sociology, professor, OSU...
Name: text_lemmatized, Length: 1100, dtype: object


In [96]:
## Let's move on for now and test these later 
## Now we will want to decrease size of data. For now, let's just go with the misrep column
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#rmp_df_sub = rmp_df.loc[:,'comment_edit','out_misrep']

### now let's split up the data into train and test sets 
# Split the data; follows the same syntax as box 6 via the commas 
X_train, X_test, y_train, y_test = train_test_split(rmp_df['text_lemmatized'], rmp_df['out_misrep'], random_state=1337,
                                                   stratify=rmp_df['out_misrep'], test_size=0.33)

# vectorizing the sentences
#cv = CountVectorizer(binary = True) # implies that it indicates whether the word is present or not.
#cv.fit(train['review']) # find all the unique words from the training set
#train_x = cv.transform(train['review'])
#test_x = cv.transform(test['review'])
X_test

285    [Dr, Boyces, class, pretty, interesting, I, ap...
303    [One, best, professor, Ive, ever, Lecture, ter...
929    [Wonderful, ProfessorVery, smartvery, friendly...
345    [Dr, Bruno, great, lecturer, His, exam, tough,...
780    [Super, nice, guy, crazymaking, quietscattered...
                             ...                        
722    [fujita, suck, get, another, professor, test, ...
233                                  [Great, instructor]
89     [Professor, Baum, fantastic, instructor, also,...
416    [Professor, Caldeiras, lecture, might, cure, n...
140    [Overall, solid, professor, Class, seem, bit, ...
Name: text_lemmatized, Length: 363, dtype: object

In [103]:
text_lemmatized_all = X_train.astype(str)
text_lemmatized_all

1043    ['Loved', 'Jungers!', 'She', 'made', 'everythi...
122     ['The', 'test', 'class', 'ridiculous', 'made',...
470     ['If', 'psychology', 'major', 'NO', 'point', '...
765     ['midterm', 'final', 'group', 'presentation', ...
2       ['Class', 'super', 'easy', 'One', 'reading', '...
                              ...                        
62      ['Really', 'great', 'guy', 'great', 'class', '...
868     ['Professor', 'Harris', 'nice', 'I', 'found', ...
36      ['Genuinely', 'interested', 'teaching', 'A', '...
342     ['I', 'dont', 'know', 'class', 'psych', 'woman...
529     ['Dr', 'Cudeck', 'brilliant', 'psychologistThe...
Name: text_lemmatized, Length: 737, dtype: object

In [112]:
### let's try to apply the above to the data
X_train = X_train.astype(str)
def tokens(x): return x.split(',')
tfidf_vect= TfidfVectorizer( tokenizer=tokens 
                            ,use_idf=True
                            , smooth_idf=True
                            , min_df = 1
                            , stop_words = 'english'
                            , max_features = 10
                            , sublinear_tf=False)

tf_idf_matrix = pd.DataFrame(
    tfidf_vect.fit_transform(X_train).toarray(), 
    columns=tfidf_vect.get_feature_names()
)

Xtrain_mat=tfidf_vect.fit_transform(X_train)
Xtrain_mat ### good, this is what we want 

C:\Users\j-curiel\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<737x10 sparse matrix of type '<class 'numpy.float64'>'
	with 1961 stored elements in Compressed Sparse Row format>

In [113]:
tf_idf_matrix ### this is a visual representation of what's being produced. Note that it is only for the first row 

,'class','easy','exam','he','i','lecture','lot','professor','really','take'
0,0.753598,0.657335,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,0.469085,0.409165,0.0,0.0,0.323676,0.712585,0.000000,0.0,0.000000,0.000000
2,0.586767,0.000000,0.0,0.0,0.809756,0.000000,0.000000,0.0,0.000000,0.000000
3,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.686385,0.0,0.0,0.000000,0.000000,0.727238,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
732,0.497311,0.867572,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
733,0.438018,0.000000,0.0,0.0,0.604479,0.665391,0.000000,0.0,0.000000,0.000000
734,0.602162,0.000000,0.0,0.0,0.000000,0.000000,0.556505,0.0,0.572454,0.000000
735,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [114]:
### now do the same for the test set 
X_test = X_test.astype(str)
Xtest_mat=tfidf_vect.fit_transform(X_test)
Xtest_mat

<363x10 sparse matrix of type '<class 'numpy.float64'>'
	with 968 stored elements in Compressed Sparse Row format>

In [121]:
### read in ordinal encorder 
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
### try to do a grid search 

xgb_train = xgb.DMatrix(Xtrain_mat, y_train, enable_categorical=True)
xgb_test = xgb.DMatrix(Xtest_mat, y_test, enable_categorical=True)


#grid_search = GridSearchCV(estimator = pipe, param_grid = param_grid, 
#                          cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

#hyperparameter fitting ; where it did not work for me ; also broke down here 
#grid_search.fit(X_train, y_train)

In [120]:
# Setting the Parameters of the Model
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

## set parameters 
params = {"objective": "multi:softprob", "tree_method": "gpu_hist", "num_class": 3}
n = 1000

results = xgb.cv(
   params, xgb_train,
   num_boost_round=n,
   nfold=3,
   metrics=["mlogloss", "auc", "merror"], ## here we are telling to look at three metrics, which are 
    # multi-class log loss, area under the ROC curve, and multi-class classification error
) # note that the auc is the flase positive rate (x axis) plotted against the true positive rate (y-axis). Closer to 1, teh 
# better, with 0 being garbage 

XGBoostError: [17:08:49] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:148: label must be in [0,1] for logistic regression

In [122]:
results.keys() # keys() method returns a list of all the keys in a dictionary, and what can be analyzed. Let's take a look


In [126]:
results['test-auc-mean'].max()# at 0.57; not ideal, but not the worst either. Better than random 



0.5726749608626386

In [127]:
misrep_model =xgb.train(params, xgb_train, num_boost_round = 30)

In [139]:
### save model 
# save to JSON
misrep_model.save_model("model.json")

## to load in: 
# misrep_model = xgb.Booster()
# misrep_model.load_model("model.json")

In [132]:
### let's apply to our test data 
y_pred = misrep_model.predict(xgb_test)
y_pred # this returns an array, which is 3 cols 

array([[9.5794606e-01, 3.6805231e-02, 5.2486709e-03],
       [9.9901891e-01, 6.8283116e-04, 2.9829287e-04],
       [9.5847327e-01, 1.9467477e-02, 2.2059243e-02],
       ...,
       [9.8856038e-01, 4.9981573e-03, 6.4414935e-03],
       [9.7794914e-01, 2.0888600e-02, 1.1622668e-03],
       [9.9745852e-01, 1.8508084e-03, 6.9073279e-04]], dtype=float32)

In [135]:
y_pred_df = pd.DataFrame(y_pred, columns=['not_misrep','somewhat_misrep','very_misrep'])
y_pred_df.head(10)

,not_misrep,somewhat_misrep,very_misrep
0,0.957946,0.036805,0.005249
1,0.999019,0.000683,0.000298
2,0.958473,0.019467,0.022059
3,0.919900,0.074875,0.005225
4,0.990735,0.004749,0.004516
5,0.912307,0.076372,0.011322
6,0.988913,0.003526,0.007562
7,0.998676,0.000720,0.000604
8,0.948045,0.048637,0.003318
9,0.997689,0.001817,0.000494


In [136]:
y_pred_df['not_misrep'].min() # lowest is still 54 %; means most are not considered misrep. 

0.545444905757904

In [143]:
### let's predict onto the total data set 
x_final = rmp_df['text_lemmatized']

### clean type 
x_final = x_final.astype(str)
x_final_mat=tfidf_vect.fit_transform(x_final)

## check 


x_final_mat = xgb.DMatrix(x_final_mat, enable_categorical=True)
x_final_mat

In [145]:
### now let's try to predict 
post_model_pred = misrep_model.predict(x_final_mat)
post_model_pred ## good 

array([[9.5847327e-01, 1.9467477e-02, 2.2059243e-02],
       [8.6221087e-01, 1.3421570e-01, 3.5734116e-03],
       [9.9688524e-01, 2.5627634e-03, 5.5202644e-04],
       ...,
       [9.1458035e-01, 7.5135954e-02, 1.0283698e-02],
       [9.5847327e-01, 1.9467477e-02, 2.2059243e-02],
       [9.9813062e-01, 7.4268936e-04, 1.1266695e-03]], dtype=float32)

In [146]:
### now create df 
post_model_pred_df = pd.DataFrame(post_model_pred, columns=['not_misrep','somewhat_misrep','very_misrep'])
post_model_pred_df.head(10)

,not_misrep,somewhat_misrep,very_misrep
0,0.958473,0.019467,0.022059
1,0.862211,0.134216,0.003573
2,0.996885,0.002563,0.000552
3,0.996198,0.001970,0.001832
4,0.989664,0.001846,0.008490
5,0.890565,0.014316,0.095119
6,0.997253,0.001418,0.001329
7,0.948045,0.048637,0.003318
8,0.998543,0.000983,0.000474
9,0.992481,0.005857,0.001663


In [147]:
### lets combine the data frame and take a look 
rmp_misrep = rmp_df.iloc[:,1:9]
rmp_misrep2 =pd.concat([rmp_misrep, post_model_pred_df], axis=1)
rmp_misrep2


,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,not_misrep,somewhat_misrep,very_misrep
0,2.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Only graded on assignments Midterm Final R...,0,0.958473,0.019467,0.022059
1,3.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Final grade is only based on two exams reading...,0,0.862211,0.134216,0.003573
2,4.0,1,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy One reading quiz a week w...,0,0.996885,0.002563,0.000552
3,4.0,2,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Lecture could be dry at times but I still like...,0,0.996198,0.001970,0.001832
4,5.0,3,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Alex was an excellent lecturer Insightful even...,0,0.989664,0.001846,0.008490
...,...,...,...,...,...,...,...,...,...,...,...
1095,4.0,4,SOC410,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes very fair I got away with calling Karl Ma...,0,0.990735,0.004749,0.004516
1096,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Professor klochko is cool and all but her acce...,0,0.918948,0.077825,0.003227
1097,1.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Definitely not the greatest thing since slice ...,0,0.914580,0.075136,0.010284
1098,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes not personal almost to a harsh point The ...,0,0.958473,0.019467,0.022059


In [137]:
## write out csv 
rmp_misrep2.to_csv("output/misrep_predicted_06212023.cs")

,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,out_emo_lang,...,pb_pol_affil,complex,constructive,reflective,outrage_agg,personal_attack_agg,prejudice_agg,comment_edit,stem,text_lemmatized
0,1,2.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Only graded on assignments Midterm Final R...,0,0.0,...,0.0,0,0,0,0.0,0,0.0,Only graded assignments Midterm Final Rea...,only graded assignments midterm final rea...,"[Only, graded, assignment, Midterm, Final, Rea..."
1,2,3.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Final grade is only based on two exams reading...,0,0.0,...,0.0,1,0,0,0.0,0,0.0,Final grade based two exams reading quizzes...,final grade based two exams reading quizzes...,"[Final, grade, based, two, exam, reading, quiz..."
2,3,4.0,1,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy One reading quiz a week w...,0,0.0,...,0.0,0,0,0,0.0,0,0.0,Class super easy One reading quiz week com...,class super easy one reading quiz week com...,"[Class, super, easy, One, reading, quiz, week,..."
3,4,4.0,2,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Lecture could be dry at times but I still like...,0,0.0,...,0.0,0,0,1,0.0,0,0.0,Lecture could dry times I still liked clas...,lecture could dry times i still liked clas...,"[Lecture, could, dry, time, I, still, liked, c..."
4,5,5.0,3,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Alex was an excellent lecturer Insightful even...,0,0.0,...,0.0,0,0,0,0.0,0,0.0,Alex excellent lecturer Insightful even hand...,alex excellent lecturer insightful even hand...,"[Alex, excellent, lecturer, Insightful, even, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,4,4.0,4,SOC410,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes very fair I got away with calling Karl Ma...,0,0.0,...,0.0,0,0,0,0.0,0,0.0,Shes fair I got away calling Karl Marx idio...,shes fair i got away calling karl marx idio...,"[Shes, fair, I, got, away, calling, Karl, Marx..."
1096,5,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Professor klochko is cool and all but her acce...,0,0.0,...,0.0,0,0,0,0.0,0,0.0,Professor klochko cool accent makes hard...,professor klochko cool accent makes hard...,"[Professor, klochko, cool, accent, make, hard,..."
1097,6,1.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Definitely not the greatest thing since slice ...,0,0.0,...,0.0,0,0,0,1.0,0,0.0,Definitely greatest thing since slice bread ...,definitely greatest thing since slice bread ...,"[Definitely, greatest, thing, since, slice, br..."
1098,7,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes not personal almost to a harsh point The ...,0,0.0,...,0.0,0,0,0,1.0,1,0.0,Shes personal almost harsh point The class ...,shes personal almost harsh point the class ...,"[Shes, personal, almost, harsh, point, The, cl..."


In [140]:
Xtest_mat

<363x10 sparse matrix of type '<class 'numpy.float64'>'
	with 968 stored elements in Compressed Sparse Row format>